In [41]:
import pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os

# Load API keys
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Create Pinecone client 
pc = Pinecone(api_key=PINECONE_API_KEY)

# Connect to existing index 
index = pc.Index("villam-chatbot")  

# Create embedding model
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GEMINI_API_KEY
)


In [39]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

# Delete the old index 
if "villam-chatbot" in pc.list_indexes().names():
    pc.delete_index("villam-chatbot")

# Create a new index with correct dimension
pc.create_index(
    name="villam-chatbot",
    dimension=768,  
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")  
)


{
    "name": "villam-chatbot",
    "metric": "cosine",
    "host": "villam-chatbot-a3uwple.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}

In [43]:
# Upload documents into Pinecone using LangChain + Pinecone
# Assumes you have `split_docs` from previous step
vectorstore = PineconeVectorStore.from_documents(
    documents=split_docs,
    embedding=embeddings,
    index_name="villam-chatbot",  
    text_key="text"
)

In [45]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [47]:
#  Define system prompt template
system_prompt_template = """
You are Villam Hub's helpful assistant. You support users in growing food, planting trees, and making sustainable choices. 
Answer the user’s question using only the following context (don’t make anything up):

{context}

Question: {question}

Give a clear, warm, and helpful answer based on the above.
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=system_prompt_template
)


In [49]:
retriever = vectorstore.as_retriever()


In [51]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # keeps it simple
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [55]:
response = qa_chain.run("what is villam?")
print(response)


GoogleGenerativeAIError: Error embedding content: Timeout of 60.0s exceeded, last exception: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: getaddrinfo: WSA Error (No such host is known.
 -- 11001)